
Example script: Alpha filtration values on a Delaunay complex.

This file demonstrates how to

- build a simplicial complex from the maximal simplices of a Delaunay triangulation
- compute squared alpha values $\alpha^2(\sigma)$ for simplices up to a chosen dimension
- verify the face monotonicity condition required for a filtration

# Mathematical conventions

## Point cloud

A point cloud is a finite subset

\begin{align}X = \{x_0,\dots,x_{N-1}\} \subset \mathbb R^d,\end{align}

represented as a NumPy array ``points`` of shape ``(N, d)``, where ``points[i] = x_i``.

## Simplices and indexing

A $k$-simplex is represented by a strictly increasing tuple of vertex indices

\begin{align}\sigma = (i_0,\dots,i_k), \qquad 0 \le i_0 < \cdots < i_k \le N-1.\end{align}

Its set of vertices is $V(\sigma) = \{x_{i_0},\dots,x_{i_k}\}$.

## Squared alpha value

For a simplex $\sigma$ with vertex set $V(\sigma)$,
its squared alpha value is the squared radius of the smallest enclosing ball of its vertices

\begin{align}\alpha^2(\sigma)
   =
   \min_{c \in \mathbb R^d}\ \max_{x\in V(\sigma)} \|x-c\|^2
   \ \in [0,\infty).\end{align}

This quantity is well defined because $V(\sigma)$ is finite.

## Alpha filtration and monotonicity

Define for each threshold $t \ge 0$ the simplicial complex

\begin{align}K_t = \{\sigma \mid \alpha^2(\sigma) \le t\}.\end{align}

For $K_t$ to be a filtration, it is necessary that $\alpha^2$ is face monotone

\begin{align}\tau \subseteq \sigma \implies \alpha^2(\tau) \le \alpha^2(\sigma).\end{align}

This script checks the inequality on the first few Delaunay triangles and their edges.


In [ ]:
from __future__ import annotations

import numpy as np

from homolipop.alpha import alpha_values_squared
from homolipop.delaunay import delaunay_triangulation
from homolipop.simplices import build_complex


def main() -> None:
    """
    Run a minimal alpha filtration demo on a fixed planar point set.

    Steps
    -----
    1. Define a small set of points in :math:`\\mathbb R^2`.
    2. Compute a Delaunay triangulation.
    3. Build the simplicial complex generated by the Delaunay triangles up to ``max_dim``.
    4. Compute squared alpha values for all simplices in that complex up to ``max_dim``.
    5. Print a small sample of simplices and their :math:`\\alpha^2` values.
    6. Check face monotonicity on triangle edges for a few triangles.

    Raises
    ------
    RuntimeError
        If face monotonicity fails on any tested triangle edge. Formally, if for some
        triangle :math:`\\sigma` and some edge :math:`\\tau \\subseteq \\sigma` we have
        :math:`\\alpha^2(\\tau) > \\alpha^2(\\sigma)` in floating point arithmetic.

    Notes
    -----
    The monotonicity check is performed on the computed floating point values.
    If you require numerically robust comparisons, incorporate a tolerance in the check.
    """
    points = np.array(
        [
            [0.0, 0.0],
            [1.0, 0.0],
            [0.0, 1.0],
            [1.0, 1.0],
            [0.3, 0.6],
        ],
        dtype=float,
    )

    delaunay = delaunay_triangulation(points)
    triangles = delaunay.delaunay_simplices

    max_dim = 2
    complex_data = build_complex(triangles, max_dim=max_dim)
    alpha = alpha_values_squared(points, triangles, max_dim=max_dim)

    print("Number of points:", len(points))
    print("Number of Delaunay triangles:", len(triangles))
    print("Number of simplices up to dim 2:", len(complex_data.all_simplices))

    print("\nSome simplices and alpha^2 values:")
    for simplex in complex_data.all_simplices[:12]:
        print(simplex, "->", alpha.alpha_sq[simplex])

    print("\nMonotonicity checks (alpha(face) <= alpha(simplex)) on first 10 triangles:")
    for tri in triangles[:10]:
        a_tri = alpha.alpha_sq[tri]
        edges = [(tri[0], tri[1]), (tri[0], tri[2]), (tri[1], tri[2])]
        for e in edges:
            if alpha.alpha_sq[e] > a_tri:
                raise RuntimeError(
                    f"monotonicity violated: {e} has {alpha.alpha_sq[e]} > {a_tri} of {tri}"
                )

    print("\nOK")


if __name__ == "__main__":
    main()